# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [19]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import gmaps
import plotly.express as px

# Import API key
from api_keys import geoapify_key 


In [20]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,khandyga,62.6667,135.6000,241.83,98,100,1.42,RU,1670189475
1,1,grindavik,63.8424,-22.4338,276.12,100,75,3.09,IS,1670189475
2,2,solnechnyy,50.7214,136.6319,248.49,89,17,1.58,RU,1670189476
3,3,bara,10.3744,10.7288,300.21,21,8,6.00,NG,1670189476
4,4,castro,-24.7911,-50.0119,292.20,96,100,1.94,BR,1670189476


### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.
* Could not get the geo api key working due to restrictions. Did some research to get the plotly working to show the similar results

In [48]:
fig = px.scatter_geo(city_data_df,
                    lat=city_data_df.Lat,
                    lon=city_data_df.Lng,
                    size= 'Humidity',
                    hover_name="City")
fig.show()

# # Configure the map plot
# map_layout = {
#      'width': '900px',
#      'height': '600px',
#      'border': '1px solid black',
#      'padding': '1px',
#      'margin': '0 auto 0 auto'
# }

# coordinates = city_data_df[['Lat', 'Lng']]
# humidity_values = city_data_df["Humidity"]

# fig = gmaps.figure(layout=map_layout,zoom_level=2,center=(15,25))

# # Display the map
# humidity_layer = gmaps.heatmap_layer(coordinates, weights=humidity_values, 
#                                  dissipating=False, max_intensity=10,
#                                  point_radius=1)

# # Add layer
# fig.add_layer(humidity_layer)

# # Display figure
# fig


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [43]:
# Narrow down cities that fit criteria and drop any results with null values
no_clouds = city_data_df.Cloudiness < 10
optimal_humid = (city_data_df.Humidity < 70 ) & (city_data_df.Humidity > 40)
less_windy = city_data_df['Wind Speed'] < 10
weather_condition = no_clouds & optimal_humid & less_windy 

# Drop any rows with null values
perfect_weather_df = city_data_df[weather_condition]
perfect_weather_df = perfect_weather_df.dropna()
perfect_weather_df = perfect_weather_df.reset_index()

# Display sample data
perfect_weather_df.head()

,index,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,14,14,bethel,41.3712,-73.4140,278.57,51,0,1.54,US,1670189261
1,18,18,jamestown,42.0970,-79.2353,274.92,44,0,6.17,US,1670189443
2,35,35,bowral,-34.4667,150.4167,292.03,45,8,3.63,AU,1670189430
3,51,51,cabo san lucas,22.8909,-109.9124,302.76,61,0,2.06,MX,1670189255
4,101,101,cape town,-33.9258,18.4232,290.52,64,0,6.17,ZA,1670189294


### Step 3: Create a new DataFrame called `hotel_df`.

In [44]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = perfect_weather_df[["City","Country","Lat","Lng","Cloudiness","Humidity","Wind Speed"]]


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

C:\Users\anand\AppData\Local\Temp\ipykernel_32756\828897073.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,City,Country,Lat,Lng,Cloudiness,Humidity,Wind Speed,Hotel Name
0,bethel,US,41.3712,-73.4140,0,51,1.54,
1,jamestown,US,42.0970,-79.2353,0,44,6.17,
2,bowral,AU,-34.4667,150.4167,8,45,3.63,
3,cabo san lucas,MX,22.8909,-109.9124,0,61,2.06,
4,cape town,ZA,-33.9258,18.4232,0,64,6.17,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [52]:
# Set parameters to search for a hotel
#https://api.geoapify.com/v2/places?categories=&conditions=wheelchair&filter=circle:-75.1651784,39.9527267,5000&bias=proximity:-75.1651784,39.9527267&limit=20&apiKey=YOUR_API_KEY
radius = 10000
params = {
    "rankby": "distance",
    "type": "restaurant",
    "categories": "catering.restaurant",
    "apiKey": geoapify_key,
    "distance": radius
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    #params["filter"] = f'circle:{lat},{lon}'
    params["bias"] = f'proximity:{lat},{lon}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
        # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params).json()
    
    # Convert the API response to JSON format
    #name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No-hotel-found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: No-hotel-found
jamestown - nearest hotel: No-hotel-found
bowral - nearest hotel: No-hotel-found
cabo san lucas - nearest hotel: No-hotel-found
cape town - nearest hotel: No-hotel-found
tasiilaq - nearest hotel: No-hotel-found
gat - nearest hotel: No-hotel-found
salvador - nearest hotel: No-hotel-found
volodarskiy - nearest hotel: Znamensk
westport - nearest hotel: No-hotel-found
xiaoshi - nearest hotel: No-hotel-found
brainerd - nearest hotel: No-hotel-found
baiyin - nearest hotel: No-hotel-found
waris aliganj - nearest hotel: No-hotel-found
talcahuano - nearest hotel: No-hotel-found
saint george - nearest hotel: No-hotel-found
winder - nearest hotel: No-hotel-found
ahipara - nearest hotel: No-hotel-found
chabahar - nearest hotel: Ravintola Kalash
ashland - nearest hotel: No-hotel-found
petropavlovsk-kamchatskiy - nearest hotel: No-hotel-found
birjand - nearest hotel: No-hotel-found
ketchikan - nearest hotel: No-hotel-found
rock sound - nea

,City,Country,Lat,Lng,Cloudiness,Humidity,Wind Speed,Hotel Name
0,bethel,US,41.3712,-73.4140,0,51,1.54,No-hotel-found
1,jamestown,US,42.0970,-79.2353,0,44,6.17,No-hotel-found
2,bowral,AU,-34.4667,150.4167,8,45,3.63,No-hotel-found
3,cabo san lucas,MX,22.8909,-109.9124,0,61,2.06,No-hotel-found
4,cape town,ZA,-33.9258,18.4232,0,64,6.17,No-hotel-found
5,tasiilaq,GL,65.6145,-37.6368,5,65,1.45,No-hotel-found
6,gat,IL,31.6100,34.7642,0,64,2.20,No-hotel-found
7,salvador,SV,13.8333,-88.9167,0,42,1.78,No-hotel-found
8,volodarskiy,RU,46.4013,48.5409,5,51,4.19,Znamensk
9,westport,US,41.1415,-73.3579,0,48,2.57,No-hotel-found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [58]:
# Configure the map plot
hotel_found_df =  hotel_df.drop(hotel_df[hotel_df['Hotel Name'] == 'No-hotel-found'].index)
hotel_found_df['hover'] = hotel_found_df['City'] + '' + hotel_found_df['Country']

fig = px.scatter_geo(hotel_found_df,
                    lat=hotel_found_df.Lat,
                    lon=hotel_found_df.Lng,
                    text=hotel_found_df['Hotel Name'])

# Display the map
fig.show()